In [12]:
from diffrax import diffeqsolve, ODETerm, Dopri5, SaveAt, LinearInterpolation, Kvaerno5
from jax import grad, jit, vmap
import jax.numpy as np
import pandas as pd
import numpy as onp

import matplotlib.pyplot as plt

In [17]:
def fomite_foi_sc(t, x, params):
    S, C, ε = x

    # p2p
    γ  = params["γ"]
    β  = params["β"]
    σ  = params["σ"]
    δ  = params["δ"]
    α  = params["δ"]
    Nw = params["Nw"]

    # fom
    θ = params["θ"]
    κ = params["κ"]

    λ1 = β * C / Nw
    λ2 = ε

    λ = λ1 + λ2

    sdot = (1-γ) * σ * Nw - λ * S - δ * S + α * C
    idot = γ * σ     + λ * S - δ * C - α * C
    εdot = θ * C / Nw - κ * ε

    return np.array([sdot, idot, εdot])


In [24]:
params_sim_df = pd.DataFrame(columns=["γ", "β", "θ", "κ"])


gammas = [5/100, 15/100, 25/100, 50/100, 75/100, 80/100]
betas  = [0.01, 0.025, 0.05]
tetas  = [1e-3, 1e-2]
kappas = [1/(1 * 30),  1/(3*30), 1/(6*30), 1/(12*30), 1/(24*30), 1/(36*30)]

idx_sim =1
for g in gammas:
    for κ in kappas:
        for b in betas:
            for θ in tetas:
                    params_sim_df = params_sim_df.append({"γ": g, "β": b, "θ": θ, "κ": κ, "sim_id": int(idx_sim)}, ignore_index=True)
                    idx_sim += 1


/var/folders/7p/jp6xqkvn5wb6ddl1fn0bhs980000gn/T/ipykernel_87135/706122452.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  params_sim_df = params_sim_df.append({"γ": g, "β": b, "θ": θ, "κ": κ, "sim_id": int(idx_sim)}, ignore_index=True)
/var/folders/7p/jp6xqkvn5wb6ddl1fn0bhs980000gn/T/ipykernel_87135/706122452.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  params_sim_df = params_sim_df.append({"γ": g, "β": b, "θ": θ, "κ": κ, "sim_id": int(idx_sim)}, ignore_index=True)
/var/folders/7p/jp6xqkvn5wb6ddl1fn0bhs980000gn/T/ipykernel_87135/706122452.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  params_sim_df = params_sim_df.append({"γ": g, "β": b, "θ": θ, "κ": κ, "sim_id": int(idx_sim)}, ignore_index=True)
/var/fo

In [22]:
import diffrax

num_years = 2
t_max     = 365 * num_years       # simulate for 5 yrs but daily
tsim      = np.arange(0, t_max+1) # list of days to simulate
num_sims  = len(params_sim_df.sim_id.unique())

solutions_arr = np.array([[],[],[],[],[],[],[],[]])
for idx_row, row in params_sim_df.iterrows():

        params = {
                "β":  row.β,
                "γ":  row.γ,
                "σ":  1/3,
                "δ":  1/3,
                "α":  1/120,
                "Nw": 1000,
                "θ" : row.θ,
                "κ" : row.κ
                }

        saveat = SaveAt(ts=tsim)
        model  = lambda t, x, args: fomite_foi_sc(t, x, params)
        term   = ODETerm(model)
        solver = Dopri5()

        stepsize_controller = diffrax.PIDController(rtol=1e-8, atol=1e-8)

        c0 = row.γ * params["σ"] * params["Nw"]
        ε0 = row.θ/row.κ *1/(params["α"] + params["δ"])
        y0 = np.array([1000-c0,  c0, ε0 ])

        solution = diffeqsolve(term, solver,
                                        t0=0, t1=t_max,
                                        saveat=saveat, dt0=1, y0=y0,
                                        stepsize_controller = stepsize_controller)

        s, c, ε       = solution.ys.T

        y = onp.concatenate([[solution.ts], [s], [c], [ε], [row.γ*np.ones(t_max+1)], [row.κ*np.ones(t_max+1)], [row.β*np.ones(t_max+1)], [row.θ*np.ones(t_max+1)] ], axis=0)
        solutions_arr = np.concatenate([solutions_arr, y], axis=1)

solution_df  = pd.DataFrame(columns=["time", "S", "C", "ε", "γ", "κ", "β", "θ"], data=solutions_arr)


XlaRuntimeError: INTERNAL: Generated function failed: CpuCallback error: RuntimeError: The maximum number of solver steps was reached. Try increasing `max_steps`.

At:
  /Users/chaosdonkey06/anaconda3/envs/pompjax/lib/python3.8/site-packages/equinox/internal/errors.py(17): raises
  /Users/chaosdonkey06/anaconda3/envs/pompjax/lib/python3.8/site-packages/jax/_src/callback.py(142): _flat_callback
  /Users/chaosdonkey06/anaconda3/envs/pompjax/lib/python3.8/site-packages/jax/_src/callback.py(42): pure_callback_impl
  /Users/chaosdonkey06/anaconda3/envs/pompjax/lib/python3.8/site-packages/jax/_src/callback.py(105): _callback
  /Users/chaosdonkey06/anaconda3/envs/pompjax/lib/python3.8/site-packages/jax/interpreters/mlir.py(1798): _wrapped_callback
  /Users/chaosdonkey06/anaconda3/envs/pompjax/lib/python3.8/site-packages/jax/interpreters/pxla.py(2113): __call__
  /Users/chaosdonkey06/anaconda3/envs/pompjax/lib/python3.8/site-packages/jax/_src/profiler.py(314): wrapper
  /Users/chaosdonkey06/an

In [ ]:
import seaborn as sns

g       = gammas[0]
fig, ax = plt.subplots(2, len(kappas), figsize=(14.5, 9.2), sharex=True, sharey=False)

for idx_k, k in enumerate(kappas):

    sim_ids_plt = list(np.squeeze(params_sim_df.query(f"γ == {g} and κ == {k}")[["sim_id"]].values))
    sol_plt_df  = solution_df[solution_df.sim_id.isin( [float(s) for s in sim_ids_plt] )]

    sns.lineplot(ax=ax[0, idx_k], x="time", y="S", data=sol_plt_df, hue="sim_id", palette="Reds")
    sns.lineplot(ax=ax[0, idx_k], x="time", y="C", data=sol_plt_df, hue="sim_id", palette="Blues")
    sns.lineplot(ax=ax[1, idx_k], x="time", y="ε", data=sol_plt_df, hue="sim_id", palette="Greens")

    ax[0, idx_k].set_title(r"$1/\kappa=${} month".format(int(1/k / 30)))

for axi in ax.flatten():
    axi.spines['right'].set_visible(False)
    axi.spines['top'].set_visible(False)
    axi.legend().remove()
